In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 


In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 49.2 MB/s 
     |████████████████████████████████| 115 kB 54.8 MB/s 
     |████████████████████████████████| 127 kB 40.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# Importando bibliotecas

In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
from datasets import load_dataset, load_metric

# Puxa dataset

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("superb", "ks")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6798 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3081 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 51094
    })
    validation: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 6798
    })
    test: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 3081
    })
})

In [ ]:

labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
import pandas as pd
data = {'file': dataset['train']['file'], 'audio': dataset['train']['audio'],
        'label':dataset['train']['label'],}
mid = pd.DataFrame.from_dict(data)
mid.head()

,file,audio,label
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10


In [ ]:
dataset.shape

{'test': (3081, 3), 'train': (51094, 3), 'validation': (6798, 3)}

In [ ]:
from sklearn.model_selection import train_test_split
perc = 10
X= mid[['file', 'audio']].copy()
y= mid['label']
X_resto, X_treino_1p, y_resto, y_treino_1p = train_test_split(X, y, stratify=y, test_size=(perc/100), random_state=42)

In [ ]:
# data = {'texto': X_treino_1p, 'label': y_treino_1p}
base_treino = pd.DataFrame(X_treino_1p, columns = ['file','audio'])
base_treino['label'] = y_treino_1p
base_treino.shape

(5110, 3)

In [ ]:
base_treino.head()

,file,audio,label
45360,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2
17229,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
41712,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
2464,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
21161,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11


In [ ]:
test = {'file': dataset['test']['file'], 'audio': dataset['test']['audio'],
        'label':dataset['test']['label'],}
test = pd.DataFrame.from_dict(test)
test.head()

,file,audio,label
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,10


In [ ]:
import pandas as pd
val = {'file': dataset['validation']['file'], 'audio': dataset['validation']['audio'],
        'label':dataset['validation']['label'],}
val = pd.DataFrame.from_dict(val)
val.head()

,file,audio,label
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,11


In [ ]:
base_treino.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_original.pkl')

In [ ]:
test.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_test.pkl')

In [ ]:
val.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_validation.pkl')

In [ ]:
import pickle


pickle.dump(label2id, open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_label2id.pkl", "wb"))  # save it into a file named save.p
pickle.dump(id2label, open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_id2label.pkl", "wb"))  # save it into a file named save.p

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
# import pickle

# favorite_color = pickle.load(open("save.p", "rb"))
# favorite_color is now {"lion": "yellow", "kitty": "red"}

# Rodando Classificador na Original

In [3]:
import pickle
label2id = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_label2id.pkl", "rb"))
id2label = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_id2label.pkl", "rb"))

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [4]:
import pandas as pd
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_original.pkl')

In [5]:
train.shape

(5110, 3)

In [7]:
train.label.value_counts(normalize=True)

11    0.636986
8     0.036986
6     0.036595
0     0.036399
9     0.036399
5     0.036204
1     0.036204
2     0.036008
7     0.036008
4     0.036008
3     0.036008
10    0.000196
Name: label, dtype: float64

In [8]:
id2label

{'0': 'yes',
 '1': 'no',
 '2': 'up',
 '3': 'down',
 '4': 'left',
 '5': 'right',
 '6': 'on',
 '7': 'off',
 '8': 'stop',
 '9': 'go',
 '10': '_silence_',
 '11': '_unknown_'}

In [ ]:
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_test.pkl')

In [ ]:
val = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_validation.pkl')

In [ ]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [ ]:
max_duration = 1.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [ ]:
encoded_train = train.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_test = test.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_val = val.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.bias', 'classifier

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=1,
    num_train_epochs=3,
    metric_for_best_model="accuracy",
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_pred):
    acc = load_metric("accuracy")
    f1 = load_metric("f1", average='macro')
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)



In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5110
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 120


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.637500,1.586072,0.620918,0.069648
2,1.523700,1.558386,0.620918,0.069648
3,1.505700,1.512749,0.620918,0.069648


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=120, training_loss=1.6494816641012828, metrics={'train_runtime': 491.7245, 'train_samples_per_second': 31.176, 'train_steps_per_second': 0.244, 'total_flos': 1.391792205888e+17, 'train_loss': 1.6494816641012828, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(encoded_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 3081
  Batch size = 32


(3081, 12) (3081,)


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
acc = load_metric("accuracy")
f1 = load_metric("f1", average='macro')



In [ ]:
accuracy = acc.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
accuracy

0.0834144758195391

In [ ]:
f1 = f1.compute(predictions=preds, references=predictions.label_ids, average="macro")["f1"]
f1

0.012832035150788894